In [7]:
# 1️⃣ Imports
# ===========================
import sys
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import arviz as az
import pymc as pm
import aesara.tensor as at
import plotly.graph_objects as go
from scipy.stats import ttest_ind

# Suppress BLAS and other numerical warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [4]:
import sys
import os

# Ensure we can import from src
sys.path.append(os.path.abspath("../src"))
from data_preprocessing import load_and_preprocess_data

In [8]:
# ===========================
# 2️⃣ Load Data
# ===========================
data_path = "../data/raw/BrentOilPrices.csv"
events_path = "../data/events.csv"

df = load_and_preprocess_data(data_path)
df['Log_Return'] = np.log(df['Price']) - np.log(df['Price'].shift(1))
data = df['Log_Return'].dropna().values
n = len(data)
idx = np.arange(n)

In [ ]:
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt
import arviz as az
import pandas as pd
import plotly.graph_objects as go
import aesara.tensor as at
from scipy.stats import ttest_ind
import warnings

# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

def load_and_preprocess_data(filepath):
    """Load and preprocess Brent oil price data."""
    try:
        df = pd.read_csv(filepath)
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
        df = df.dropna()
        print(f"Loaded data with {len(df)} records")
        return df
    except Exception as e:
        raise ValueError(f"Data loading failed: {e}")

def quantify_event_impact(df, change_index, window=30):
    """Quantify impact of change point on oil prices."""
    start = max(0, change_index - window)
    end = min(len(df) - 1, change_index + window)

    before = df['Price'].iloc[start:change_index]
    after = df['Price'].iloc[change_index:end]

    return {
        "mean_change": after.mean() - before.mean(),
        "volatility_change": after.std() - before.std(),
        "pct_price_change": ((after.mean() - before.mean()) / before.mean()) * 100,
        "t_test_pvalue": ttest_ind(before, after, equal_var=False).pvalue
    }

def bayesian_multiple_change_point_model(filepath: str, events_filepath: str, k=3):
    """Bayesian model with k change points for Brent oil data."""
    try:
        # Load and preprocess data
        df = load_and_preprocess_data(filepath)
        df['Log_Return'] = np.log(df['Price']) - np.log(df['Price'].shift(1))
        data = df['Log_Return'].dropna().values
        n = len(data)
        
        if len(data) < k + 1:
            raise ValueError(f"Data length ({len(data)}) too short for {k} change points.")
        
        print(f"Analyzing {n} data points with {k} change points")

        with pm.Model() as model:
            # Define k ordered change points
            tau = pm.Uniform(
                "tau",
                lower=np.zeros(k),
                upper=np.ones(k)*n,
                shape=k,
                transform=pm.distributions.transforms.Ordered(),
                initval=np.linspace(n*0.2, n*0.8, k)
            )
            
            # Segment parameters
            mu = pm.Normal("mu", mu=0, sigma=1, shape=k+1)
            sigma = pm.HalfNormal("sigma", sigma=1)
            
            # Piecewise mean construction using switch
            segment_means = at.zeros(n)
            for i in range(k+1):
                if i == 0:
                    condition = at.lt(at.arange(n), tau[0])
                elif i == k:
                    condition = at.ge(at.arange(n), tau[-1])
                else:
                    condition = at.and_(at.ge(at.arange(n), tau[i-1]),
                                      at.lt(at.arange(n), tau[i]))
                
                segment_means = at.set_subtensor(
                    segment_means[condition],
                    mu[i]
                )
            
            # Likelihood
            obs = pm.Normal("obs", mu=segment_means, sigma=sigma, observed=data)
            
            # Sampling
            trace = pm.sample(
                1000,
                tune=1000,
                target_accept=0.9,
                return_inferencedata=True,
                cores=1
            )
            
            # Posterior predictive
            ppc = pm.sample_posterior_predictive(trace, var_names=["obs"])

        # Analysis and visualization
        analyze_results(trace, df, ppc, events_filepath, k)
        
        return trace, df

    except Exception as e:
        print(f"Model execution failed: {str(e)}")
        raise

def analyze_results(trace, df, ppc, events_filepath, k):
    """Analyze and visualize the model results."""
    # Extract change points
    tau_samples = trace.posterior["tau"].values
    detected_taus = np.mean(tau_samples, axis=(0, 1)).astype(int)
    detected_dates = df['Date'].iloc[detected_taus].values

    print("\n📌 Detected Change Points:")
    for i, (idx, date) in enumerate(zip(detected_taus, detected_dates)):
        impact = quantify_event_impact(df, idx)
        print(f"\nChange point {i+1} on {date.date()}:")
        print(f"  • Mean change: {impact['mean_change']:.4f}")
        print(f"  • Volatility change: {impact['volatility_change']:.4f}")
        print(f"  • Price change: {impact['pct_price_change']:.2f}%")
        print(f"  • Statistical significance (p-value): {impact['t_test_pvalue']:.4f}")

    # Create plot
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(df['Date'], df['Price'], label='Brent Price')
    
    for i, (idx, date) in enumerate(zip(detected_taus, detected_dates)):
        ax.axvline(x=date, color='red', linestyle='--', alpha=0.7)
        ax.text(date, df['Price'].max(), f"CP {i+1}", 
                ha='center', va='bottom', color='red')
    
    ax.set_title(f"Brent Oil Price with {k} Detected Change Points")
    ax.set_xlabel("Date")
    ax.set_ylabel("Price (USD)")
    ax.legend()
    plt.show()

    # Model diagnostics
    print("\n🔍 Model Diagnostics:")
    print(az.summary(trace, var_names=["tau", "mu", "sigma"]))
    
    az.plot_trace(trace, var_names=["tau", "mu", "sigma"])
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    try:
        trace, df = bayesian_multiple_change_point_model(
            r"C:\10x AIMastery\brent-oil-change-analysis\data\raw\BrentOilPrices.csv",
            r"C:\10x AIMastery\brent-oil-change-analysis\data\events.csv",
            k=3
        )
    except Exception as e:
        print(f"Execution failed: {str(e)}")

NotImplementedError: Cannot convert spacings to a tensor variable.